In [31]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

Experimento: FE + valores negativos tomo promedio 12 meses.

Kaggle = 0.282


##### Levantamos el dataset con target ya calculado

In [2]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

##### Preprocesamiento a la minima expresión :)

In [3]:
##### aplicamos OHE
df = preprocesamiento.aplicarOHE(df)
df.shape

(31362, 162)

### Feature Engineering

##### Neural Prophet

In [4]:
neural_prophet_fe = pd.read_csv("./datasets/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 165)

##### Prophet

In [5]:
prophet_fe = pd.read_csv("./datasets/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 171)

##### FE Moviles

In [6]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
# df = feature_engineering.get_rolling_medians(df, "tn", 201912)
# df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
# df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 626)

##### FE Moviles sobre otras variables

In [7]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

# #  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty_zscore')

##### FE Calendario

In [8]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 651)

##### FE sobre FE

In [9]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 680)

##### Variables Exogenas

In [10]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 683)

##### EXP 1

-   


In [11]:
study, models, results = model_lgb.main_entrenar(df, VERSION="v12")

[I 2025-06-26 22:15:29,066] A new study created in memory with name: no-name-21f62f9e-b45a-45b6-86e4-c4e9a601e22d


Features: ['product_id', 'periodo', 'nacimiento_producto', 'muerte_producto', 'mes_n', 'total_meses', 'producto_nuevo', 'ciclo_de_vida_inicial', 'sku_size', 'stock_final', 'tn', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'cat1_FOODS', 'cat1_HC', 'cat1_PC', 'cat1_REF', 'cat1_nan', 'cat2_ADEREZOS', 'cat2_CABELLO', 'cat2_DENTAL', 'cat2_DEOS', 'cat2_HOGAR', 'cat2_OTROS', 'cat2_PIEL1', 'cat2_PIEL2', 'cat2_PROFESIONAL', 'cat2_ROPA ACONDICIONADOR', 'cat2_ROPA LAVADO', 'cat2_ROPA MANCHAS', 'cat2_SOPAS Y CALDOS', 'cat2_TE', 'cat2_VAJILLA', 'cat2_nan', 'cat3_ACONDICIONADOR', 'cat3_Acond Bebe', 'cat3_Acond Mujer', 'cat3_Aero', 'cat3_Aji Picante', 'cat3_Almidon', 'cat3_Antigrasa', 'cat3_Azul', 'cat3_BARRAS', 'cat3_Baking Bags', 'cat3_Barbacoa', 'cat3_Barra', 'cat3_Baño', 'cat3_Bizcochuelo', 'cat3_CREMA', 'cat3_CUIDADO ESPECIAL', 'cat3_Caldo Cubo', 'cat3_Caldos Granulados', 'cat3_Cara', 'cat3_Cepilllo', 'cat3_Chimichurri', 'cat3_Concentrado', 'cat3_Cremoso', 'cat3_Cristalino', 

[I 2025-06-26 22:17:27,481] Trial 0 finished with value: 15.59830462682434 and parameters: {'max_depth': 9, 'learning_rate': 0.2536999076681772, 'n_estimators': 759, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352}. Best is trial 0 with value: 15.59830462682434.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[27]	valid_0's l1: 26.1512
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[31]	valid_0's l1: 28.7795
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[28]	valid_0's l1: 25.1314
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[49]	valid_0's l1: 14.4094
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[50]	valid_0's l1: 14.4708
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[47]	valid_0's l1: 14.4938
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[53]	valid_0's l1: 12.1418
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[118]	valid_0's l1: 12.1514
Tr

[I 2025-06-26 22:18:24,661] Trial 1 finished with value: 15.64481160015418 and parameters: {'max_depth': 13, 'learning_rate': 0.11114989443094977, 'n_estimators': 118, 'min_child_weight': 10, 'subsample': 0.9162213204002109, 'colsample_bytree': 0.6061695553391381, 'reg_alpha': 0.18182496720710062, 'reg_lambda': 0.18340450985343382}. Best is trial 0 with value: 15.59830462682434.


...

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[31]	valid_0's l1: 33.1954
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[72]	valid_0's l1: 31.7913
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[23]	valid_0's l1: 29.4275
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[50]	valid_0's l1: 14.2193
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[50]	valid_0's l1: 14.6992
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[48]	valid_0's l1: 14.4128
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[120]	valid_0's l1: 11.2272
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[159]	valid_0's l1: 11.5128
Training until

[I 2025-06-27 01:13:40,991] Trial 48 finished with value: 16.2614951510172 and parameters: {'max_depth': 17, 'learning_rate': 0.10244215332627643, 'n_estimators': 893, 'min_child_weight': 10, 'subsample': 0.977538264840095, 'colsample_bytree': 0.8887177203699564, 'reg_alpha': 0.19998346486682367, 'reg_lambda': 0.9999590939522055}. Best is trial 45 with value: 12.069536772395356.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[150]	valid_0's l1: 17.2185
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[111]	valid_0's l1: 18.3416
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[137]	valid_0's l1: 17.3062
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[128]	valid_0's l1: 13.492
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[120]	valid_0's l1: 13.4501
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[124]	valid_0's l1: 13.2315
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[175]	valid_0's l1: 10.489
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[188]	valid_0's l1: 10.1805
Training u

[I 2025-06-27 01:18:06,994] Trial 49 finished with value: 12.147985477221408 and parameters: {'max_depth': 14, 'learning_rate': 0.05654435777828026, 'n_estimators': 866, 'min_child_weight': 1, 'subsample': 0.9416843983685306, 'colsample_bytree': 0.8875278728669203, 'reg_alpha': 0.13111978413888947, 'reg_lambda': 0.8274027056937}. Best is trial 45 with value: 12.069536772395356.



🏆 Mejores parámetros:
max_depth: 17
learning_rate: 0.05890388302112587
n_estimators: 936
min_child_weight: 1
subsample: 0.9644425566679116
colsample_bytree: 0.8874372579769764
reg_alpha: 0.289959077780367
reg_lambda: 0.9107294720478487
boosting_type: gbdt
objective: regression
metric: mae
verbosity: -1
deterministic: True
force_col_wise: True

Entrenando modelo final...
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[936]	valid_0's l1: 2.44808
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[936]	valid_0's l1: 2.41836
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[936]	valid_0's l1: 2.37154
Ensemble MAE Val: 2.3305

🎯 MAE Test Ensemble: 2.4000

Top 5 mejores trials:
1. MAE: 12.0695
2. MAE: 12.0958
3. MAE: 12.0961
4. MAE: 12.0979
5. MAE: 12.1235


In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Supongamos que querés predecir para los períodos 202011 y 202012
periodos_future = [201911, 201912]
df_future = df[df['periodo'].isin(periodos_future)].copy()

# Asegurate de tener las mismas columnas que en entrenamiento
features = [col for col in df.columns if col not in ['target']]

# Predecir
future_preds = model_lgb.predict_with_ensemble(models, df_future, features)

# Si querés evaluar el MAE (solo si tenés target en esos períodos)
if 'target' in df_future:
    mae_future = mean_absolute_error(df_future['target'], future_preds)
    print(f"📊 MAE Future: {mae_future:.4f}")

# Guardar las predicciones
df_future['pred_ensemble'] = future_preds

📊 MAE Future: 30.4279


In [13]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df_future = df_future[df_future['periodo'] == 201912]
df_future = df_future[df_future['product_id'].isin(productos_ok['product_id'].unique())]
df_future = df_future[['product_id', 'pred_ensemble']]
df_future.rename(columns={'pred_ensemble': 'tn'}, inplace=True)



In [16]:
df_future[df_future['tn'] < 0]

,product_id,tn
30814,20381,-0.970889
30947,20565,-0.884263
30982,20612,-3.205840
30988,20622,-1.484349
31009,20652,-3.768568
...,...,...
31353,21259,-1.886019
31354,21262,-2.179109
31355,21263,-2.380752
31357,21265,-0.635106


In [ ]:
promedios = model_lgb.promedio_12_meses_780p()
df_future = df_future.merge(promedios, on='product_id', how='left')


,product_id,tn_x,tn_y
0,20001,1485.164526,1454.732720
1,20002,1073.127766,1175.437142
2,20003,632.648987,784.976407
3,20004,512.758038,627.215328
4,20005,638.582322,668.270104
...,...,...,...
775,21263,-2.380752,0.029993
776,21265,-0.635106,0.089541
777,21266,1.782685,0.094659
778,21267,-3.725477,0.092835


In [23]:
df_future.rename(columns={'tn_x': 'tn', 'tn_y': 'tn_12m'}, inplace=True)
df_future['tn'] = df_future['tn'].mask(df_future['tn'] < 0, df_future['tn_12m'])
df_future

,product_id,tn,tn_12m
0,20001,1485.164526,1454.732720
1,20002,1073.127766,1175.437142
2,20003,632.648987,784.976407
3,20004,512.758038,627.215328
4,20005,638.582322,668.270104
...,...,...,...
775,21263,0.029993,0.029993
776,21265,0.089541,0.089541
777,21266,1.782685,0.094659
778,21267,0.092835,0.092835


In [24]:
df_future.drop(columns=['tn_12m'], inplace=True)
df_future.to_csv("./outputs/predicciones_exp_04_lgb_fe_v1.csv", index=False, sep=',')

Feature importance

In [26]:
model_lgb.feature_importance(df, models, "04")

In [34]:
importances = model_lgb.feature_importance_promedio(models, df.drop(columns=['target']))
importances.sort_values(by='importance', ascending=False, inplace=True) 
importances

,feature,importance
165,yearly_add,0.023933
650,tn_expanding_mean,0.020740
12,cust_request_qty,0.020621
658,tn_seasonality,0.019636
670,tn_seasonal_naive,0.019149
...,...,...
622,tn_rolling_max_34,0.000000
623,tn_rolling_max_35,0.000000
624,tn_rolling_max_36,0.000000
25,cat2_PIEL1,0.000000
